In [1]:
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline

In [3]:
import glob

timestamps = []
all_raw_dfs = {}
num_samples = 0

for f in glob.glob('rtt/*.csv'):
    t = f[8:-4]
    timestamps.append(t)
    print("Reading", f)
    df = pd.read_csv(f).set_index('domain')
    all_raw_dfs[t] = df
    num_samples += 1
print("Total: ", num_samples)

Reading rtt/rtt-1614650401.csv
Reading rtt/rtt-1614762001.csv
Reading rtt/rtt-1615096801.csv
Reading rtt/rtt-1614639601.csv
Reading rtt/rtt-1614934801.csv
Reading rtt/rtt-1614924001.csv
Reading rtt/rtt-1614873601.csv
Reading rtt/rtt-1615294801.csv
Reading rtt/rtt-1614783601.csv
Reading rtt/rtt-1615413601.csv
Reading rtt/rtt-1614952801.csv
Reading rtt/rtt-1615244401.csv
Reading rtt/rtt-1614884401.csv
Reading rtt/rtt-1615028401.csv
Reading rtt/rtt-1615384801.csv
Reading rtt/rtt-1615305601.csv
Reading rtt/rtt-1614877201.csv
Reading rtt/rtt-1614664801.csv
Reading rtt/rtt-1614672001.csv
Reading rtt/rtt-1615039201.csv
Reading rtt/rtt-1615377601.csv
Reading rtt/rtt-1614906001.csv
Reading rtt/rtt-1615309201.csv
Reading rtt/rtt-1615478401.csv
Reading rtt/rtt-1615392001.csv
Reading rtt/rtt-1615057201.csv
Reading rtt/rtt-1614938401.csv
Reading rtt/rtt-1615093201.csv
Reading rtt/rtt-1615464001.csv
Reading rtt/rtt-1615118401.csv
Reading rtt/rtt-1615237201.csv
Reading rtt/rtt-1615215601.csv
Reading 

### Client Processing Time

In [4]:
all_client_proc = []

for (_, df) in all_raw_dfs.items():
    all_client_proc = all_client_proc + df['client_proc'].to_list()

t_client_proc = pd.Series(all_client_proc).describe(percentiles=[.90, .95, .99])["90%"]
print(pd.Series(all_client_proc).describe(percentiles=[.90, .95, .99]), "\nt_client_proc:", t_client_proc)

count    27601.000000
mean         0.983835
std          0.325436
min          0.215893
50%          0.910367
90%          1.179783
95%          1.624153
99%          2.055975
max         13.875268
dtype: float64 
t_client_proc: 1.179783


## RTT

In [5]:
all_raw_dfs[timestamps[0]].head()

,id,rtt_tlse,rtt_tcpping,client_proc
domain,,,,
www.westminster.edu,106206,4.830111,1.718826,0.899205
www.lincoln.edu,155445,8.452459,1.815011,0.721304
www.american.edu,14394,8.320986,3.485811,1.129556
www.com.edu,217283,11.195927,1.886032,0.726984
gallaudet.edu,93736,13.532343,3.352763,0.863984


In [6]:
anchors_df = pd.read_csv('../anchors-nv.csv')
domains = anchors_df['domain'].to_list()
print(len(domains))

116


In [7]:
# bad_domains = pd.read_csv('../bad-anchors.csv')['domain'].to_list()
# domains = set(domains) - set(bad_domains)
# print(len(domains))

In [8]:
# domain_df = pd.DataFrame(columns=[['time', 'rtt_tlse', 'rtt_tcpping', 'client_proc']])
l = {}
num_no_replies = {}
for domain in domains:
    l[domain] = []
    num_no_replies[domain] = 0

for (t, df) in all_raw_dfs.items():
    for domain in domains:
        if domain not in df.index:
            # l[domain].append([int(t), 50, 50, 0])
            num_no_replies[domain] += 1
            # print(domain, t, "Non existent")
            continue
        # print(df.loc['www.westminster.edu'].to_list()[1:-1])
        t_server_proc = df.at[domain, 'rtt_tlse'] - (df.at[domain, 'rtt_tcpping'] + df.at[domain, 'client_proc'])
        l[domain].append([int(t)] + df.loc[domain].to_list()[1:] + [t_server_proc])

domain_dfs = {}
for domain in domains:
    domain_dfs[domain] = pd.DataFrame(l[domain], columns=['time', 't_tlsrtt', 't_rawrtt', 't_clientproc', 't_serverproc']).set_index('time')

In [9]:
domain_dfs['www.sunysuffolk.edu'].describe(percentiles=[.5, .75, .90, .95])

,t_tlsrtt,t_rawrtt,t_clientproc,t_serverproc
count,238.000000,238.000000,238.000000,238.000000
mean,39.786972,11.765758,0.878658,27.142556
std,70.895607,9.844173,0.102293,69.451074
min,13.171547,8.529497,0.784862,-3.130981
50%,15.225043,9.328052,0.853360,5.021697
75%,23.676870,11.557557,0.890821,7.884161
90%,58.280305,16.185165,0.945276,13.285752
95%,232.849980,17.056214,0.999585,222.658448
max,496.975766,132.849792,1.744775,487.521771


In [10]:
l = {}
unreliable = []

for domain in domains:
    if num_no_replies[domain] > 5:
        print("Skipping", domain, 100 * num_no_replies[domain] / num_samples, "percent no responses")
        unreliable.append(domain)
        continue
    stat_df = domain_dfs[domain].describe(percentiles=[.5, .75, .90])
    l[domain] = [stat_df.at['mean', 't_tlsrtt'], stat_df.at['std', 't_tlsrtt'], 
                 stat_df.at['75%', 't_tlsrtt'], stat_df.at['75%', 't_serverproc'],
                 stat_df.at['50%', 't_tlsrtt'], stat_df.at['50%', 't_serverproc'],
                 stat_df.at['min', 't_rawrtt'], stat_df.at['50%', 't_rawrtt']]

print(len(l), "domains remain after removing unreliable ones")
mean_df = pd.DataFrame.from_dict(l, orient='index', columns=['mean', 'std', '75%', 'serverproc_75', '50%', 'serverproc_50', 'rawrtt_min', 'rawrtt_50'])

Skipping esu.edu 73.33333333333333 percent no responses
Skipping trustees.ufl.edu 10.416666666666666 percent no responses
Skipping usf.edu 33.75 percent no responses
113 domains remain after removing unreliable ones


In [11]:
mean_df[['mean', 'std', '75%', '50%']].sort_values('75%').round(4).to_csv('actual_rtt.csv')
# mean_df[['mean', 'std', 'server_proc_90', 'server_proc_90']].sort_values('90%').round(4).to_csv('actual_rtt.csv')

In [12]:
# print(mean_df['serverproc_75'].describe(percentiles=[.5, .75, .90, .95]))
print(mean_df['serverproc_50'].describe(percentiles=[.5, .75, .90, .95]))
t_serverproc = mean_df['serverproc_50'].describe(percentiles=[.5, .75, .90, .95])['75%']

count    113.000000
mean       5.139961
std        4.453573
min        0.731095
50%        3.653757
75%        6.186016
90%        9.335405
95%       12.813115
max       33.468443
Name: serverproc_50, dtype: float64


In [13]:
domains1 = [domain for domain in domains if domain not in unreliable and mean_df.at[domain, 'serverproc_50'] < t_serverproc]
print("Removing", set(domains) - set(domains1))
len(domains1) # 75%

Removing {'www.mass.edu', 'www.atlm.edu', 'www.com.edu', 'www.siumed.edu', 'www.etsu.edu', 'www.osumc.edu', 'www.niu.edu', 'www.gordon.edu', 'www.passhe.edu', 'www.washburn.edu', 'www.southern.edu', 'www.musc.edu', 'rice.edu', 'fordham.edu', 'www.uga.edu', 'usf.edu', 'www.usergioarboleda.edu.co', 'www.umsl.edu', 'www.imsa.edu', 'esu.edu', 'www.hgtc.edu', 'www.uwf.edu', 'uncg.edu', 'www.mc3.edu', 'www.queens.edu', 'bju.edu', 'www.eksu.edu.ng', 'wisc.edu', 'www.lincoln.edu', 'www.einstein.edu', 'trustees.ufl.edu', 'umflint.edu'}


84

In [14]:
from math import cos, asin, sqrt

# In km
def calcDistance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return round(12742 * asin(sqrt(a)), 2) #2*R*asin...


In [15]:
locations_df = pd.read_csv('../location/ip2location.csv').set_index('domain')

In [16]:
expected_rtt = {}

awsnv_lat = 38.747650
awsnv_lon = -77.532448

speedup = 3
l = []
adv_speed = (2.0/3)*299.792458
t_start = 5

for domain in domains1:
    if domain not in locations_df.index:
        print(domain, " location unknown")
    d = calcDistance(awsnv_lat, awsnv_lon, locations_df.at[domain, 'lat'], locations_df.at[domain, 'lon'])
    rtt_upper_bound = (2*d) / adv_speed
    rtt_lower_bound = rtt_upper_bound * speedup + t_start
    l.append([domain, rtt_upper_bound, rtt_lower_bound, mean_df.at[domain, 'rawrtt_min'], mean_df.at[domain, 'rawrtt_50']])

expected_df = pd.DataFrame(l, columns=['domain', 'rtt_adv', 'rtt_honest', 'observed_min', 'observed_50']).set_index('domain')
expected_df['speedup'] = expected_df['observed_min'] / expected_df['rtt_adv']
print(len(expected_df))
expected_df.head()

84


,rtt_adv,rtt_honest,observed_min,observed_50,speedup
domain,,,,,
www.westminster.edu,31.297752,98.893256,0.944682,1.529478,0.030184
www.american.edu,0.440205,6.320614,2.777052,2.990575,6.308549
gallaudet.edu,0.499245,6.497736,3.051308,3.527633,6.111840
www.usuhs.edu,0.464922,6.394765,3.430243,4.177223,7.378110
www.ucmerced.edu,10.987301,37.961903,1.328771,1.583906,0.120937


In [17]:
expected_df[expected_df['speedup'] <= 1.25]
# expected_df[expected_df['observed_50'] >= expected_df['rtt_honest']]

,rtt_adv,rtt_honest,observed_min,observed_50,speedup
domain,,,,,
www.westminster.edu,31.297752,98.893256,0.944682,1.529478,0.030184
www.ucmerced.edu,10.987301,37.961903,1.328771,1.583906,0.120937
www.pic.edu.pk,67.647299,207.941897,20.742832,22.007894,0.306632
www.asds.edu.co,39.303891,122.911672,34.866780,35.275320,0.887108


In [18]:
print("Removing", len(expected_df[expected_df['speedup'] <= 1.25]))
expected_df = expected_df[expected_df['speedup'] > 1.25]

print("Removing", len(expected_df[expected_df['observed_50'] >= expected_df['rtt_honest']]))

expected_df = expected_df[expected_df['observed_50'] < expected_df['rtt_honest']]
len(expected_df)

Removing 4
Removing 20


60

In [19]:
import math

t_porcom = 2
expected_df['error (km)'] = (expected_df['rtt_honest'] + t_client_proc + t_serverproc + t_porcom)*(adv_speed / 2)
expected_df['amplification'] = (1000 / (expected_df['rtt_honest'] + t_client_proc + t_serverproc + t_porcom))
expected_df['amplification'] = expected_df['amplification'].apply(np.floor)
expected_df['amplification'] -= 1
print("client_proc:", t_client_proc)
print("server_proc:", t_serverproc)
expected_df.sort_values('rtt_adv')

client_proc: 1.179783
server_proc: 6.186016499999999


,rtt_adv,rtt_honest,observed_min,observed_50,speedup,error (km),amplification
domain,,,,,,,
www.american.edu,0.440205,6.320614,2.777052,2.990575,6.308549,1567.556114,62.0
www.usuhs.edu,0.464922,6.394765,3.430243,4.177223,7.378110,1574.966114,62.0
gallaudet.edu,0.499245,6.497736,3.051308,3.527633,6.111840,1585.256114,62.0
www.kutztown.edu,2.485920,12.457759,8.654655,9.189196,3.481470,2180.846114,44.0
www.pointpark.edu,2.831859,13.495577,6.581955,7.354089,2.324252,2284.556114,42.0
www.lasu.edu.ng,3.311791,14.935373,7.836681,8.264238,2.366297,2428.436114,40.0
www.sunysuffolk.edu,4.506017,18.518052,8.529497,9.328052,1.892913,2786.456114,34.0
qu.edu,4.567560,18.702680,13.137940,13.949549,2.876359,2804.906114,34.0
www.quinnipiac.edu,4.567560,18.702680,13.133572,13.971397,2.875402,2804.906114,34.0


In [20]:
# roughtime.sandbox.google.com

t_rt_proc = 2

print("domain | rtt_adv | rtt_honest | Delta | error (km)")
for domain in ["roughtime.chainpoint.org", "roughtime.sandbox.google.com", "roughtime.int08h.com"]:
    d = calcDistance(awsnv_lat, awsnv_lon, locations_df.at[domain, 'lat'], locations_df.at[domain, 'lon'])
    rtt_upper_bound = (2*d) / adv_speed
    rtt_lower_bound = rtt_upper_bound * speedup + t_start
    Delta = rtt_lower_bound + t_client_proc + t_rt_proc + t_porcom
    error = Delta * adv_speed / 2
    print(domain, rtt_upper_bound, rtt_lower_bound, Delta, error)

domain | rtt_adv | rtt_honest | Delta | error (km)
roughtime.chainpoint.org 0.46031845137344984 6.380955354120349 11.56073835412035 1155.274055825538
roughtime.sandbox.google.com 1.1540984129760863 8.46229523892826 13.64207823892826 1363.2640558255382
roughtime.int08h.com 15.839257704074729 52.51777311222419 57.69755611222419 5765.764055825538
